
运行环境: Python36_Computing  
内容： 实验 Least Square Rigid Motion using SVD  
准备简单实验数据  
1. 为了简单明了的验证，生成一个简单的正方形“点云” （0,0,0）,（1,0,0), (1,1,0), (0,1,0) 用一个3x4 的矩阵表示
2. 利用算转矩阵和平移矩阵，将P逆时针旋转45度，向x正方向平移2

来自Paper：Sorkine-Hornung, O., Rabinovich, M.: Least-squares rigid motion using svd. no. 3, 1–5 (2017)



In [1]:
import numpy as np

In [2]:
# 给定旋转角度
theta = 45
theta = (theta / 180.0) * np.pi
print("Radian: %.3f" % theta)

Radian: 0.785


In [3]:
# 点云P
P = np.array([ [0, 0, 0],
               [1, 0, 0],
               [1, 1, 0],
               [0, 1, 0]])
P = P.T  # 变为列矩阵 ！！！！

real_rotation = np.array([ [np.cos(theta), -np.sin(theta), 0],
                           [np.sin(theta), np.cos(theta), 0],
                           [0, 0,1]
                           ])

real_t = np.array([2,0,0]).reshape(3,1)
Q = np.around(np.dot(real_rotation, P)) + real_t  # 由于P用列矩阵表示，这里的矩阵乘法很方便

## 1. 计算$\overline{p} 和 \overline{q}$
$$\overline{p} = \frac{\Sigma_{i=1}^{n}w_ip_i}{\Sigma_{i=1}^{n}w_i}$$  
$$\overline{q} = \frac{\Sigma_{i=1}^{n}w_iq_i}{\Sigma_{i=1}^{n}w_i}$$

In [4]:
def compute_centroid(X):
    centroid = np.zeros(3).reshape(3,1)
    for i in range(X.shape[1]):
        centroid += X[:, i].reshape(3,1)
    centroid = centroid / X.shape[1]
    return centroid

p_centroid = compute_centroid(P)
q_centroid = compute_centroid(Q)

## 2. 计算中心向量(centered vectors)
$$x_i := p_i - \overline{p}, \qquad y_i := q_i - \overline{q}, \qquad i=1,2,\ldots,n$$

In [5]:
# 矩阵表示
X = P-p_centroid
Y = Q-q_centroid

## 3. 计算$d\times d$的协方差矩阵(covariance matrix)
$$S = XWY^T$$
$X$ 和 $Y$ 的维度是 $d\times n$, $W=diag(w_1, w_2, \ldots, w_n)$. 这里$n=4$.


In [6]:
W = np.identity(4) * 0.25  # 权重设置，因为这里实验中总共有4个点，所以直接平均了，每个点的权重是0.25
S = np.dot(np.dot(X,W), Y.T)

## 4. 对S做奇异值分解（SVD）得到$S=U\Sigma V^T$
则旋转矩阵为：
$$\begin{equation}
R
=V\begin{bmatrix}
1  &     &  \ & \\
   &  1  &  \ &  \\
     &   &    &    \\
   &    &  \ & det(VU^{T})\\
\end{bmatrix}
U^T
\end{equation}
$$


In [7]:
u, s, vh = np.linalg.svd(S, full_matrices=False)

C = np.identity(3)
C[-1,-1] = np.linalg.det(np.dot(vh,u.T))

R = np.dot(np.dot(vh,C),u.T)
print("计算得到的旋转矩阵")
print(R)
print("真实旋转矩阵")
print(real_rotation)

计算得到的旋转矩阵
[[ 0.70710678 -0.70710678  0.        ]
 [ 0.70710678  0.70710678  0.        ]
 [ 0.          0.          1.        ]]
真实旋转矩阵
[[ 0.70710678 -0.70710678  0.        ]
 [ 0.70710678  0.70710678  0.        ]
 [ 0.          0.          1.        ]]


## 5.计算平移向量 $t$
$$t=\overline{q} - R\overline{p}$$

In [8]:
t = q_centroid - np.dot(R, p_centroid)
print("计算得到的平移向量")
print(t)
print("真实平移向量")
print(real_t)

计算得到的平移向量
[[2.        ]
 [0.04289322]
 [0.        ]]
真实平移向量
[[2]
 [0]
 [0]]


In [9]:
# 1. 计算结果和真实值还是有差距的，应该是由于浮点数的计算误差所导致的
# 2. 使用列矩阵 真的 很好，比如计算R*P, R是3x3, P是3x4， 可以直接np.dot(R,P)得到一个3x4的矩阵，每一列表示一个点